In [ ]:
import pandas as pd
import zipfile
import os
from google.colab import files as colab_files

# Load the Excel file
file_path = '/content/June_Break Time-Part 2 (1).xls'
xls = pd.ExcelFile(file_path)

# Function to split tables and include the two lines above each table
def split_tables(df):
    tables = []
    current_table = None
    current_headers = []

    for idx, row in df.iterrows():
        if pd.isna(row[0]) and pd.isna(row[1]):
            # Encounter a header row
            if current_table is not None:
                tables.append((current_headers, current_table))
            current_headers = [row.tolist()]
            current_table = pd.DataFrame()
        elif current_headers and not pd.isna(row[0]):
            # Add rows to the current table
            current_table = pd.concat([current_table, pd.DataFrame([row.tolist()])], ignore_index=True)
        else:
            current_headers.append(row.tolist())

    if current_table is not None:
        tables.append((current_headers, current_table))

    return tables

# Read the first sheet
df = xls.parse(xls.sheet_names[0])

# Split the tables
tables = split_tables(df)

# Create a directory to save the split tables
output_dir = '/content/split_tables'
os.makedirs(output_dir, exist_ok=True)

# Save each table as a separate Excel file
for i, (headers, table) in enumerate(tables):
    file_name = f'table_{i+1}.xlsx'
    file_path = os.path.join(output_dir, file_name)

    with pd.ExcelWriter(file_path) as writer:
        # Write headers
        for j, header in enumerate(headers):
            header_df = pd.DataFrame([header])
            header_df.to_excel(writer, index=False, header=False, startrow=j)
        # Write table
        table.to_excel(writer, index=False, header=True, startrow=len(headers))

# Create a zip file containing all the split tables
zip_file_path = '/content/split_tables.zip'
with zipfile.ZipFile(zip_file_path, 'w') as zipf:
    for root, _, files in os.walk(output_dir):
        for file in files:
            zipf.write(os.path.join(root, file), file)

# Download the zip file
colab_files.download(zip_file_path)

print("All tables have been split and saved into a zip file successfully.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

All tables have been split and saved into a zip file successfully.
